In [1]:
import numpy as np

# Функция сигмоиды


def sigmoid(x):
    # f(x) = 1 / (1 + e^(-x))
    return 1 / (1 + np.exp(-x))

# Производная сигмоиды


def deriv_sigmoid(x):
    # f'(x) = f(x) * (1 - f(x))
    fx = sigmoid(x)
    return fx * (1 - fx)

# Функция потерь MSE


def mse_loss(y_true, y_pred):
    # y_true и y_pred - массивы numpy одинаковой длины.
    return ((y_true - y_pred) ** 2).mean()


class OurNeuralNetwork:
    '''
    Нейронная сеть с:
    - 2 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходным слоем с 1 нейроном (o1)
    '''

    def __init__(self):
        # Веса
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        # Смещения
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()

    def feedforward(self, x):
        # x - массив numpy с 2 элементами.
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1

    def train(self, data, all_y_trues):
        '''
        - data - (n x 2) массив numpy, n = количество образцов в наборе данных.
        - all_y_trues - массив numpy с n элементами.
          Элементы в all_y_trues соответствуют тем в data.
        '''
        learn_rate = 0.1
        epochs = 1000  # количество раз, когда мы пройдемся по всему набору данных

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # --- Выполняем прямой проход (нам понадобятся эти значения позже)
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1

                # --- Вычисляем частные производные.
                # --- Название: p_L_p_w1 означает "частная производная L по w1"
                p_L_p_ypred = -2 * (y_true - y_pred)

                # Нейрон o1
                p_ypred_p_w5 = h1 * deriv_sigmoid(sum_o1)
                p_ypred_p_w6 = h2 * deriv_sigmoid(sum_o1)
                p_ypred_p_b3 = deriv_sigmoid(sum_o1)

                p_ypred_p_h1 = self.w5 * deriv_sigmoid(sum_o1)
                p_ypred_p_h2 = self.w6 * deriv_sigmoid(sum_o1)

                # Нейрон h1
                p_h1_p_w1 = x[0] * deriv_sigmoid(sum_h1)
                p_h1_p_w2 = x[1] * deriv_sigmoid(sum_h1)
                p_h1_p_b1 = deriv_sigmoid(sum_h1)

                # Нейрон h2
                p_h2_p_w3 = x[0] * deriv_sigmoid(sum_h2)
                p_h2_p_w4 = x[1] * deriv_sigmoid(sum_h2)
                p_h2_p_b2 = deriv_sigmoid(sum_h2)

                # --- Обновляем веса и смещения
                # Нейрон h1
                self.w1 -= learn_rate * p_L_p_ypred * p_ypred_p_h1 * p_h1_p_w1
                self.w2 -= learn_rate * p_L_p_ypred * p_ypred_p_h1 * p_h1_p_w2
                self.b1 -= learn_rate * p_L_p_ypred * p_ypred_p_h1 * p_h1_p_b1

                # Нейрон h2
                self.w3 -= learn_rate * p_L_p_ypred * p_ypred_p_h2 * p_h2_p_w3
                self.w4 -= learn_rate * p_L_p_ypred * p_ypred_p_h2 * p_h2_p_w4
                self.b2 -= learn_rate * p_L_p_ypred * p_ypred_p_h2 * p_h2_p_b2

                # Нейрон o1
                self.w5 -= learn_rate * p_L_p_ypred * p_ypred_p_w5
                self.w6 -= learn_rate * p_L_p_ypred * p_ypred_p_w6
                self.b3 -= learn_rate * p_L_p_ypred * p_ypred_p_b3

            # --- Вычисляем общую потерю в конце каждой эпохи
            if epoch % 10 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print("Потеря эпохи %d: %.3f" % (epoch, loss))

    def predict(self, x):
        # Используем метод feedforward для получения выхода последнего нейрона
        output = self.feedforward(x)
        # Округляем выход до ближайшего целого числа
        rounded_output = np.round(output)
        return rounded_output


# Определение набора данных
data = np.array([
    [-2, -1],  # Алиса
    [25, 6],   # Боб
    [17, 4],   # Чарли
    [-15, -6],  # Диана
])
all_y_trues = np.array([
    1,  # Алиса
    0,  # Боб
    0,  # Чарли
    1,  # Диана
])

# Обучение нашей нейронной сети!
network = OurNeuralNetwork()
network.train(data, all_y_trues)

# Делаем предсказания
emily = np.array([-7, -3])  # 128 фунтов, 63 дюйма
frank = np.array([20, 2])  # 155 фунтов, 68 дюймов
print("Эмили: %.3f" % network.predict(emily))  # 0.951 - F
print("Франк: %.3f" % network.predict(frank))  # 0.039 - M

Потеря эпохи 0: 0.394
Потеря эпохи 10: 0.293
Потеря эпохи 20: 0.185
Потеря эпохи 30: 0.121
Потеря эпохи 40: 0.086
Потеря эпохи 50: 0.064
Потеря эпохи 60: 0.050
Потеря эпохи 70: 0.041
Потеря эпохи 80: 0.034
Потеря эпохи 90: 0.029
Потеря эпохи 100: 0.025
Потеря эпохи 110: 0.022
Потеря эпохи 120: 0.020
Потеря эпохи 130: 0.018
Потеря эпохи 140: 0.016
Потеря эпохи 150: 0.015
Потеря эпохи 160: 0.013
Потеря эпохи 170: 0.012
Потеря эпохи 180: 0.011
Потеря эпохи 190: 0.011
Потеря эпохи 200: 0.010
Потеря эпохи 210: 0.009
Потеря эпохи 220: 0.009
Потеря эпохи 230: 0.008
Потеря эпохи 240: 0.008
Потеря эпохи 250: 0.008
Потеря эпохи 260: 0.007
Потеря эпохи 270: 0.007
Потеря эпохи 280: 0.007
Потеря эпохи 290: 0.006
Потеря эпохи 300: 0.006
Потеря эпохи 310: 0.006
Потеря эпохи 320: 0.006
Потеря эпохи 330: 0.005
Потеря эпохи 340: 0.005
Потеря эпохи 350: 0.005
Потеря эпохи 360: 0.005
Потеря эпохи 370: 0.005
Потеря эпохи 380: 0.005
Потеря эпохи 390: 0.004
Потеря эпохи 400: 0.004
Потеря эпохи 410: 0.004
Пот

Имя	    Вес (в фунтах)	Рост (в дюймах)	Пол
Анна	133 (54.4 кг)	65 (165,1 см)	Ж
Сергей  160 (65,44 кг)	72 (183 см)     М
Макс	152 (62.2 кг)	70 (178 см)	    М
Ольга	120 (49 кг)	60 (152 см)      	Ж
Дарья	133 (54.4 кг)	65 (165,1 см)	Ж
Роман   160 (65,44 кг)	72 (183 см)	    М
Виктор	152 (62.2 кг)	70 (178 см)	    М
Ирина	120 (49 кг)	60 (152 см)      	Ж
Катя	133 (54.4 кг)	65 (165,1 см)	Ж
Саша    160 (65,44 кг)	72 (183 см)	    М



In [5]:
# Создание 10 новых примеров данных
new_data = np.array([
    [-2, -1],
    [25, 6],
    [17, 4],
    [-15, -6],
    [-2, -1],
    [25, 6],
    [17, 4],
    [-15, -6],
    [-2, -1],
    [25, 6],
])

# Предположим, что у нас есть истинные значения для этих примеров
true_values = np.array([
    1,
    0,
    0,
    1,
    1,
    0,
    0,
    1,
    1,
    0
])

# Получение предсказаний на новых примерах
predictions = np.array([network.predict(x) for x in new_data])

# Сравнение предсказаний с истинными значениями
correct_predictions = np.sum(predictions == true_values)
total_predictions = len(true_values)
accuracy = correct_predictions / total_predictions

print("Точность предсказаний: %.2f%%" % (accuracy * 100))

Точность предсказаний: 100.00%
